In [1]:
import sys
import os
import nbimporter

# 外皮情報LV4からLV5へのコンバート

## 1. 外皮性能計算プログラム＋簡単な追加情報

- 用途別の外皮、内壁の情報が得られている状態より、以下を実施する。  
　　床フラグの付与  
　　壁体（開口部以外の不透明部位）の応答係数の算出  
　　複数の壁体、部位の応答係数を集約  

### 1.1 Floor Flag

#### Outline

以下に該当する部位を、床と判定する。  
　方位が'Bottom'である外皮の部位(set_floor_flag)  
　土間床（make_earthfloor）  
　床下空間上の床（set_floor_flag_innerwall）  
　室上の床（set_floor_flag_innerwall）  
【要確認】方位が'Bottom'である外皮の部位、土間床の合計面積が、用途別床面積を超過する場合にどうするか

#### Function

In [2]:
def set_floor_flag(direction):     
    return True if direction == 'Bottom' else False

#### Example

In [3]:
print(set_floor_flag('Top'))
print(set_floor_flag('Bottom'))

False
True


### 1.2 Responce Factor

#### Ro, Ri

以下に基づき、表面熱伝達抵抗を与える。

平成28年省エネルギー基準に準拠したエネルギー消費性能の評価に関する技術情報（住宅）  
　２．エネルギー消費性能の算定方法  
　　2.2　算定方法  
　　　第三章　 暖冷房負荷と外皮性能  
　　　　第三節　熱貫流率及び線熱貫流率  
　　　　　付録 A 住宅の平均熱貫流率算出に用いる建材等の熱物性値等   
　　　　　　表 3.1 表面熱伝達抵抗   
　　　　　　表 3.2 表面熱伝達抵抗（界壁・界床の場合） 

<div style="text-align: center;"> 表3.1 表面熱伝達抵抗 </div>

| 部位 | 熱的境界内側（室内側）の<br>表面熱伝達抵抗[㎡K/W] | 熱的境界外側（外気側）の表面熱伝達抵抗[㎡K/W]<br>外気に直接接する場合 | 熱的境界外側（外気側）の表面熱伝達抵抗[㎡K/W]<br>左記以外の場合 |
|---|---|---|---|
| 屋根 | 0.09 | 0.04 | 0.09（通気層等） |
| 天井 | 0.09 | - | 0.09（小屋裏等） |
| 外壁 | 0.11 | 0.04 | 0.11（通気層等） |
| 床 | 0.15 | 0.04 | 0.15(床裏等) |

<div style="text-align: center;"> 表3.2 表面熱伝達抵抗（界壁・界床の場合） </div>

| 部位 | 対象住戸の室内側表面熱伝達抵抗[㎡K/W] | 隣接住戸の室内側表面熱伝達抵抗[㎡K/W] |
|---|---|---|
| 界壁 | 0.11 | 0.11 |
| 上階側界床 | 0.09 | 0.09 |
| 下階側界床 | 0.15 | 0.15 |

In [4]:
def get_Ro_Ri(type, IsInContactWithOutsideAir):
    
    if IsInContactWithOutsideAir == True :
        Ro = {
            'Roof'            : 0.040,
            'Ceiling'         : 0.040,
            'Wall'            : 0.040,
            'Floor'           : 0.040
        }[type]
    else :
        Ro = {
            'Roof'            : 0.090,
            'Ceiling'         : 0.090,
            'Wall'            : 0.110,
            'Floor'           : 0.150,
            'BoundaryCeiling' : 0.090,
            'BoundaryWall'    : 0.110,
            'BoundaryFloor'   : 0.150
        }[type]
        
    Ri = {
        'Roof' : 0.090,
        'Ceiling' : 0.090, 
        'Wall' : 0.110, 
        'Floor' : 0.150, 
        'BoundaryCeiling' : 0.090,
        'BoundaryWall' : 0.110,
        'BoundaryFloor'   : 0.150
    }[type]
    
    return Ro, Ri

In [5]:
def get_Ro_Ri_innerwall(type):
    
    Ro = {
        'InnerCeiling'    : 0.150,
        'InnerWall'       : 0.110,
        'InnerFloor'      : 0.090,
        'GroundFloor'     : 0.090,
    }[type]
        
    Ri = {
        'InnerCeiling'    : 0.090,
        'InnerWall'       : 0.110,
        'InnerFloor'      : 0.150,
        'GroundFloor'     : 0.150,
    }[type]
    
    return Ro, Ri

#### 熱貫流率

以下に基づき、一般部位の熱貫流率を算出する。

平成28年省エネルギー基準に準拠したエネルギー消費性能の評価に関する技術情報（住宅）  
　２．エネルギー消費性能の算定方法  
　　2.2　算定方法  
　　　第三章　 暖冷房負荷と外皮性能  
　　　　第三節　熱貫流率及び線熱貫流率  

#### 詳細計算法

$$
\displaystyle
U_{i}= \sum_{k}{a_{i,k} \times U_{i,k}}
$$

$U_{i}$: 一般部位$i$の熱貫流率, W/(m<sup>2</sup>・K)  
$a_{i,k}$: 一般部位$i$の部位$k$の面積比率, -  
$U_{i,k}$: 一般部位$i$の部位$k$の熱貫流率, W/(m<sup>2</sup>・K)  

In [6]:
def calc_UA_from_details(type, IsInContactWithOutsideAir, Parts):
    
    # 平均熱貫流率の算出
    Ro, Ri = get_Ro_Ri(type, IsInContactWithOutsideAir)
    UA = sum(x['AreaRatio']/(Ro + sum(y['thick']/y['cond'] for y in x['Layers']) + Ri) for x in Parts)
    
    return UA

In [7]:
d = {
    'Walls' : [
        {
            'direction' : 'ClosedBackFloor',
            'IsInContactWithOutsideAir': True,
            'structure' : 'Wood', 
            'type' : 'Floor',
            'InputMethod' : 'InputAllDetails',
            'Parts' :  [{'AreaRatio' : 0.80, 'Layers' : [{'cond': 0.16, 'name': 'PED', 'specH': 720.0, 'thick': 0.012}]},
                        {'AreaRatio' : 0.20, 'Layers' : [{'cond': 0.16, 'name': 'PED', 'specH': 720.0, 'thick': 0.012}]}] 
        }
    ]
}

def get_UA_from_details(d):
    for x in d['Walls'] :
        if x['InputMethod'] == 'InputAllDetails' :
            x['UA'] = calc_UA_from_details(x['type'], x['IsInContactWithOutsideAir'], x['Parts'])
    return d

print(get_UA_from_details(d))

{'Walls': [{'direction': 'ClosedBackFloor', 'IsInContactWithOutsideAir': True, 'structure': 'Wood', 'type': 'Floor', 'InputMethod': 'InputAllDetails', 'Parts': [{'AreaRatio': 0.8, 'Layers': [{'cond': 0.16, 'name': 'PED', 'specH': 720.0, 'thick': 0.012}]}, {'AreaRatio': 0.2, 'Layers': [{'cond': 0.16, 'name': 'PED', 'specH': 720.0, 'thick': 0.012}]}], 'UA': 3.7735849056603774}]}


#### 面積比率法（充填断熱する場合又は充填断熱し付加断熱する場合）（簡略計算方法①） 

$$
\displaystyle
U_{i}= \sum_{k}{a_{i,k} \times U_{i,k}}
$$

$U_{i}$: 一般部位$i$の熱貫流率, W/(m<sup>2</sup>・K)  
$a_{i,k}$: 一般部位$i$の部位$k$の面積比率, -  
$U_{i,k}$: 一般部位$i$の部位$k$の熱貫流率, W/(m<sup>2</sup>・K)  

#### 木造における床の面積比率

<div style="text-align: center;"> 表3-1 木造における床の面積比率 </div>

| 工法の種類等 | 断熱部分 | 熱橋部分 |
|---|---|---|
| 軸組構法・床梁工法（根太間に断熱）  | 0.80 | 0.20 |
| 軸組構法・束立大引工法（根太間に断熱）  | 0.80 | 0.20 |
| 軸組構法・束立大引工法（大引間に断熱）  | 0.85 | 0.15 |
| 軸組構法・束立大引工法（根太間及び大引間に断熱）  |  |  |
| 軸組構法・剛床工法  | 0.85 | 0.15 |
| 軸組構法・床梁土台同面工法（根太間に断熱）  | 0.70 | 0.30 |
| 枠組壁工法（根太間に断熱）  | 0.87 | 0.13 |

<div style="text-align: center;"> 表3-2 軸組構法の束立大引工法において根太間及び大引間に断熱する場合の床の面積比率 </div>

| 工法の種類等 | 断熱部分<br>（根太間断熱材＋大引間断熱材） | 断熱部分＋熱橋部分<br>（根太間断熱材＋大引材等） | 断熱部分＋熱橋部分<br>（根太材＋大引間断熱材） | 熱橋部分<br>（根太材＋大引材等） |
|---|---|---|---|---|
|  軸組構法の束立大引工法において<br>根太間及び大引間に断熱する場合 | 0.72 | 0.12 | 0.13 | 0.03 |

In [8]:
def get_area_ratio_wood_floor(FloorConstructionMethod):
    # FloorConstructionMethod: 床の工法種類
    # 'FrameBeamInsuljoist'（軸組構法・床梁工法（根太間に断熱））/'FrameFootingInsuljoist'（軸組構法・束立大引工法（根太間に断熱））/
    # 'FrameFootingInsulsleeper'（軸組構法・束立大引工法（大引間に断熱））/
    # 'FrameFootingInsuljoistSleeper'（軸組構法・束立大引工法（根太間及び大引間に断熱））/
    # 'FrameRigidfloor'（軸組構法・剛床工法）/'FrameBeambaseInsuljoist'（軸組構法・床梁土台同面工法（根太間に断熱））/
    # 'WallInsuljoist'（枠組壁工法（根太間に断熱））
    
    if FloorConstructionMethod == 'FrameBeamInsuljoist' :
        area_ratio_wood_floor = {'Insulation' : 0.80, 'Heatbridge' : 0.20}
    elif FloorConstructionMethod == 'FrameFootingInsuljoist' :
        area_ratio_wood_floor = {'Insulation' : 0.80, 'Heatbridge' : 0.20}
    elif FloorConstructionMethod == 'FrameFootingInsulsleeper' :
        area_ratio_wood_floor = {'Insulation' : 0.85, 'Heatbridge' : 0.15}
    elif FloorConstructionMethod == 'FrameFootingInsuljoistSleeper' :
        area_ratio_wood_floor = {
            'InsulJoitandsleeper' : 0.72,
            'InsuljoistHeatbridgesleeper' : 0.12,
            'InsulsleeperHeatbridgejoist' : 0.13,
            'HeatbridgeJoistandsleeper' : 0.03
        }
    elif FloorConstructionMethod == 'FrameRigidfloor' :
        area_ratio_wood_floor = {'Insulation' : 0.85, 'Heatbridge' : 0.15}
    elif FloorConstructionMethod == 'FrameBeambaseInsuljoist' :
        area_ratio_wood_floor = {'Insulation' : 0.70, 'Heatbridge' : 0.30}
    elif FloorConstructionMethod == 'WallInsuljoist' :
        area_ratio_wood_floor = {'Insulation' : 0.87, 'Heatbridge' : 0.13}
    else :
        raise ValueError
    
    return area_ratio_wood_floor

#### 木造における外壁の面積比率

<div style="text-align: center;"> 表4-1 木造における外壁（界壁）の面積比率 </div>

| 工法の種類等 | 断熱部分 | 熱橋部分 |
|---|---|---|
| 軸組構法・柱、間柱間に断熱 | 0.83 | 0.17 |
| 軸組構法・柱、間柱間に断熱し付加断熱 |  |  |
| 枠組壁工法・たて枠間に断熱 | 0.77 | 0.23 |
| 枠組壁工法・たて枠間に断熱＋付加断熱 |  |  |

<div style="text-align: center;"> 表4-2 軸組構法において柱・間柱間に断熱し付加断熱する場合の外壁の面積比率 </div>

| 工法の種類等 | 断熱部分<br>（柱・間柱間断熱材＋付加断熱材） | 断熱部分＋熱橋部分<br>（柱・間柱間断熱材＋付加断熱層内熱橋部分） | 断熱部分＋熱橋部分<br>（構造部材等＋付加断熱材） | 熱橋部分<br>（構造部材等＋付加断熱層内熱橋部分） |
|---|---|---|
| 付加断熱層内熱橋<br>部分が「横下地」 | 0.75 | 0.08 | 0.12 | 0.05 |
| 付加断熱層内熱橋<br>部分が「縦下地」 | 0.79 | 0.04 | 0.04 | 0.13 |

<div style="text-align: center;"> 表4-3 枠組壁工法においてたて枠間に断熱し付加断熱する場合の外壁の面積比率 </div>

| 工法の種類等 | 断熱部分<br>（充填断熱材＋付加断熱材） | 断熱部分＋熱橋部<br>（充填断熱材＋付加断熱層内熱橋部） | 断熱部分＋熱橋部<br>（構造部材等＋付加断熱材） | 断熱部分＋熱橋部<br>（まぐさ＋付加断熱材） | 熱橋部分<br>（構造部材等＋付加断熱層内熱橋部） | 熱橋部分<br>（まぐさ＋付加断熱層内熱橋部） |
|---|---|---|
| 付加断熱層内熱橋部分が「横下地」 | 0.69 | 0.08 | 0.14 | 0.02 | 0.06 | 0.01 |
| 付加断熱層内熱橋部分が「縦下地」 | 0.76 | 0.01 |  | 0.02 | 0.20 | 0.01 |

In [9]:
def get_area_ratio_wood_wall(WallConstructionMethod):
    # WallConstructionMethod: 床の工法種類
    # 'FrameInsulcolumn'（軸組構法・柱、間柱間に断熱）/
    # 'FrameInsuladdBackhorizontal'（軸組構法・柱、間柱間に断熱し付加断熱（付加断熱層内熱橋部分が「横下地」））/
    # 'FrameInsuladdBackvertical'（軸組構法・柱、間柱間に断熱し付加断熱（付加断熱層内熱橋部分が「縦下地」））/
    # 'WallInsuljamb'（枠組壁工法・たて枠間に断熱）/
    # 'WallInsuladdBackhorizontal'（枠組壁工法・たて枠間に断熱＋付加断熱（付加断熱層内熱橋部分が「横下地」））/
    # 'WallInsuladdBackvertical'（枠組壁工法・たて枠間に断熱＋付加断熱（付加断熱層内熱橋部分が「縦下地」））
    
    if WallConstructionMethod == 'FrameInsulcolumn' :
        area_ratio_wood_wall = {'Insulation' : 0.83, 'Heatbridge' : 0.17}
    elif WallConstructionMethod == 'FrameInsuladdBackhorizontal' :
        area_ratio_wood_wall = {
            'FrameInsulColumnandadd' : 0.75,
            'FrameInsulcolumnHeatbridgeadd' : 0.08,
            'FrameInsuladdHeatbridgemember' : 0.12,
            'FrameHeatbridgeMemberandadd' : 0.05
        }
    elif WallConstructionMethod == 'FrameInsuladdBackvertical' :
        area_ratio_wood_wall = {
            'FrameInsulColumnandadd' : 0.79,
            'FrameInsulcolumnHeatbridgeadd' : 0.04,
            'FrameInsuladdHeatbridgemember' : 0.04,
            'FrameHeatbridgeMemberandadd' : 0.13
        }
    elif WallConstructionMethod == 'WallInsuljamb' :
        area_ratio_wood_wall = {'Insulation' : 0.77, 'Heatbridge' : 0.23}
    elif WallConstructionMethod == 'WallInsuladdBackhorizontal' :
        area_ratio_wood_wall = {
            'WallInsulFilledandadd' : 0.69,
            'WallInsulfilledHeatbridgeadd' : 0.08,
            'WallInsuladdHeatbridgemember' : 0.14,
            'WallInsuladdHeatbridgelintel' : 0.02,
            'WallHeatbridgeMemberandadd' : 0.06,
            'WallHeatbridgeLintelandadd' : 0.01
        }
    elif WallConstructionMethod == 'WallInsuladdBackvertical' :
        area_ratio_wood_wall = {        
            'WallInsulFilledandadd' : 0.76,
            'WallInsulfilledHeatbridgeadd' : 0.01,
            'WallInsuladdHeatbridgemember' : 0.00,
            'WallInsuladdHeatbridgelintel' : 0.02,
            'WallHeatbridgeMemberandadd' : 0.20,
            'WallHeatbridgeLintelandadd' : 0.01
        }
    else :
        raise ValueError
    
    return area_ratio_wood_wall

#### 木造における天井の面積比率

<div style="text-align: center;"> 表5 木造における天井の面積比率 </div>

| 工法の種類等 | 断熱部分 | 熱橋部分 |
|---|---|---|
| 桁・梁間に断熱する場合 | 0.87 | 0.13 |

In [10]:
def get_area_ratio_wood_ceiling(CeilingConstructionMethod):
    # CeilingConstructionMethod: 天井の工法種類
    # 'Insulbeam'（桁・梁間に断熱）
    
    if CeilingConstructionMethod == 'Insulbeam' :
        area_ratio_wood_ceiling = {'Insulation' : 0.83, 'Heatbridge' : 0.17}
    else :
        raise ValueError

    return area_ratio_wood_ceiling

#### 木造における屋根の面積比率

<div style="text-align: center;"> 表6-1 木造における屋根の面積比率 </div>

| 工法の種類等 | 断熱部分 | 熱橋部分 |
|---|---|---|
| たるき間に断熱する場合  | 0.86 | 0.14 |

<div style="text-align: center;"> 表6-2 木造においてたるき間に断熱し付加断熱（横下地）する場合の屋根の面積比率  </div>

| 工法の種類等 | 断熱部分<br>（たる木間断熱材＋付加断熱材） | 断熱部分＋熱橋部分<br>（たる木間断熱材＋付加断熱層内熱橋部（下地たる木）） | 断熱部分＋熱橋部分<br>（構造部材＋付加断熱材） | 熱橋部分<br>（構造部材＋付加断熱層内熱橋部（下地たる木）） |
|---|---|---|---|---|
| たるき間に断熱し付加断熱（横下地）する場合  | 0.79 | 0.08 | 0.12 | 0.01 |

In [11]:
def get_area_ratio_wood_roof(RoofConstructionMethod):
    # RoofConstructionMethod : 屋根の工法種類
    # 'Insulrafter'（たるき間に断熱）/'InsuladdBackhorizontal'（たるき間に断熱＋付加断熱（横下地）））
    
    if RoofConstructionMethod == 'Insulrafter' :
        area_ratio_wood_roof = {'Insulation' : 0.86, 'Heatbridge' : 0.14}
    elif RoofConstructionMethod == 'InsuladdBackhorizontal' :
        area_ratio_wood_roof = {
            'InsulRafterandadd' : 0.79,
            'InsulrafterHeatbridgeadd' : 0.08,
            'InsuladdHeatbridgemember' : 0.12,
            'HeatbridgeMemberandadd' : 0.01
        }
    else :
        raise ValueError
    
    return area_ratio_wood_roof

In [12]:
def get_area_ratio_wood(type, RoofConstructionMethod = None, CeilingConstructionMethod = None, 
                        WallConstructionMethod = None, FloorConstructionMethod = None, **kwargs):
    
    if type == 'Roof' or type == 'Ceiling' :
        return get_area_ratio_wood_roof(RoofConstructionMethod)
    elif type == 'BoundaryCeiling' :
        return get_area_ratio_wood_ceiling(CeilingConstructionMethod)        
    elif type == 'Wall' or type == 'BoundaryWall' :
        return get_area_ratio_wood_wall(WallConstructionMethod)            
    elif type == 'Floor' :
        return get_area_ratio_wood_floor(FloorConstructionMethod)    
    else :
        raise ValueError        

In [13]:
def calc_UA_from_layers(type, IsInContactWithOutsideAir, Parts, RoofConstructionMethod = None, CeilingConstructionMethod = None, 
                        WallConstructionMethod = None, FloorConstructionMethod = None, **kwargs):
    
    # 面積比率の取得
    area_ratio =  get_area_ratio_wood(type, RoofConstructionMethod, CeilingConstructionMethod, WallConstructionMethod, FloorConstructionMethod)

    for x in Parts :
        if type == 'Roof' or type == 'Ceiling' :
            x['AreaRatio'] = area_ratio[x['TypeRoof']]    
        elif type == 'BoundaryCeiling' :
            x['AreaRatio'] = area_ratio[x['TypeCeiling']]
        elif type == 'Wall' or type == 'BoundaryWall' :
            x['AreaRatio'] = area_ratio[x['TypeWall']]
        elif type == 'Floor' or type == 'BoundaryFloor' :
            x['AreaRatio'] = area_ratio[x['TypeFloor']]

    # 平均熱貫流率の算出
    Ro, Ri = get_Ro_Ri(type, IsInContactWithOutsideAir)
    UA = sum(x['AreaRatio']/(Ro + sum(y['thick']/y['cond'] for y in x['Layers']) + Ri) for x in Parts)
    
    return UA

#### Example

In [14]:
d = {
    'Walls' : [
        {
            'area': 10,
            'direction' : 'Bottom',
            'IsInContactWithOutsideAir': True,
            'name' : 'Floor',
            'space' : 'Main',
            'nextspace' : 'ClosedBackFloor',
            'structure' : 'Wood', 
            'type' : 'Floor',
            'InputMethod' : 'InputAllLayers',
            'FloorConstructionMethod' : 'FrameFootingInsuljoist',
            'Parts' :  [
                {'TypeFloor': 'Insulation', 'Layers': [{'cond': 0.16, 'name': 'PED', 'specH': 720.0, 'thick': 0.012}]},
                {'TypeFloor': 'Heatbridge', 'Layers': [{'cond': 0.16, 'name': 'PED', 'specH': 720.0, 'thick': 0.012}]}
            ]
        },
        {
            'area': 20,
            'direction' : 'Top',
            'IsInContactWithOutsideAir': True,
            'name' : 'Roof',
            'space' : 'Main',
            'nextspace' : 'OutsideAir',
            'structure' : 'Wood', 
            'type' : 'Roof',
            'InputMethod' : 'InputAllLayers',
            'RoofConstructionMethod' : 'InsuladdBackhorizontal',
            'Parts' :  [
                {'TypeRoof': 'InsulRafterandadd', 'Layers' : [{'cond': 0.16, 'name': 'PED', 'specH': 720.0, 'thick': 0.012}]},
                {'TypeRoof': 'InsulrafterHeatbridgeadd', 'Layers' : [{'cond': 0.16, 'name': 'PED', 'specH': 720.0, 'thick': 0.012}]},
                {'TypeRoof': 'InsuladdHeatbridgemember', 'Layers' : [{'cond': 0.16, 'name': 'PED', 'specH': 720.0, 'thick': 0.012}]},
                {'TypeRoof': 'HeatbridgeMemberandadd', 'Layers' : [{'cond': 0.16, 'name': 'PED', 'specH': 720.0, 'thick': 0.012}]}
            ]
        },
        {
            'area': 30,
            'direction' : 'North',
            'IsInContactWithOutsideAir': True,
            'name' : 'Wall',
            'space' : 'Main',
            'structure' : 'Wood', 
            'nextspace' : 'OutsideAir',
            'type' : 'Wall',
            'InputMethod' : 'InputAllLayers',
            'WallConstructionMethod' : 'WallInsuladdBackvertical',
            'Parts' :  [
                {'TypeWall': 'WallInsulFilledandadd', 'Layers': [{'cond': 0.16, 'name': 'PED', 'specH': 720.0, 'thick': 0.012}]},
                {'TypeWall': 'WallInsulfilledHeatbridgeadd', 'Layers': [{'cond': 0.16, 'name': 'PED', 'specH': 720.0, 'thick': 0.012}]},
                {'TypeWall': 'WallInsuladdHeatbridgemember', 'Layers': [{'cond': 0.16, 'name': 'PED', 'specH': 720.0, 'thick': 0.012}]},
                {'TypeWall': 'WallInsuladdHeatbridgelintel', 'Layers': [{'cond': 0.16, 'name': 'PED', 'specH': 720.0, 'thick': 0.012}]},
                {'TypeWall': 'WallHeatbridgeMemberandadd', 'Layers': [{'cond': 0.16, 'name': 'PED', 'specH': 720.0, 'thick': 0.012}]},
                {'TypeWall': 'WallHeatbridgeLintelandadd', 'Layers': [{'cond': 0.16, 'name': 'PED', 'specH': 720.0, 'thick': 0.012}]}
            ]
        }
    ]
}

def get_UA_from_layers(d):
    for x in d['Walls'] :
        if x['InputMethod'] == 'InputAllLayers' :
            x['UA'] = calc_UA_from_layers(**x)
    return d

print(get_UA_from_layers(d))

{'Walls': [{'area': 10, 'direction': 'Bottom', 'IsInContactWithOutsideAir': True, 'name': 'Floor', 'space': 'Main', 'nextspace': 'ClosedBackFloor', 'structure': 'Wood', 'type': 'Floor', 'InputMethod': 'InputAllLayers', 'FloorConstructionMethod': 'FrameFootingInsuljoist', 'Parts': [{'TypeFloor': 'Insulation', 'Layers': [{'cond': 0.16, 'name': 'PED', 'specH': 720.0, 'thick': 0.012}], 'AreaRatio': 0.8}, {'TypeFloor': 'Heatbridge', 'Layers': [{'cond': 0.16, 'name': 'PED', 'specH': 720.0, 'thick': 0.012}], 'AreaRatio': 0.2}], 'UA': 3.7735849056603774}, {'area': 20, 'direction': 'Top', 'IsInContactWithOutsideAir': True, 'name': 'Roof', 'space': 'Main', 'nextspace': 'OutsideAir', 'structure': 'Wood', 'type': 'Roof', 'InputMethod': 'InputAllLayers', 'RoofConstructionMethod': 'InsuladdBackhorizontal', 'Parts': [{'TypeRoof': 'InsulRafterandadd', 'Layers': [{'cond': 0.16, 'name': 'PED', 'specH': 720.0, 'thick': 0.012}], 'AreaRatio': 0.79}, {'TypeRoof': 'InsulrafterHeatbridgeadd', 'Layers': [{'con

#### 熱貫流率補正法（簡略計算方法②）

<div style="text-align: center;"> 表7 木造における一般部位の断熱工法等に応じた補正熱貫流率 </div>

| 部位 | 断熱工法等 | 軸組構法補正熱貫流率 | 枠組壁構法補正熱貫流率 |
|---|---|---|---|
| 床 |  | 0.13 | 0.08 |
| 上階側界床 |  | 0.13 | 0.08 |
| 下階側界床 |  | 0.13 | 0.08 |
| 外壁・界壁 | 充填断熱（柱・間柱間に断熱）する場合 | 0.09 | 0.13 |
| 外壁・界壁 | 充填断熱（柱・間柱間に断熱）し付加断熱する場合 | 0.09 | 0.13 |

| 部位 | 断熱工法等 | 補正熱貫流率 |
|---|---|---|
| 外壁・界壁 | 土壁で外張断熱の場合 | 0.04 |
| 外壁・界壁 | 土壁以外で外張断熱の場合 | 0.02 |
| 天井 | 桁・梁間に断熱する場合 | 0.05 |
| 屋根 | 充填断熱（たるき間に断熱）する場合 | 0.11 |
| 屋根 | 充填断熱（たるき間に断熱）し付加断熱する場合 | 0.11 |
| 屋根 | 外張断熱工法 | 0.02 |

In [15]:
def get_Ur_wood(type, ConstructionMethod = None, RoofInsulationMethod = None, WallInsulationMethod = None, **kwargs):
    # Type : 部位種別
    # ConstructionMethod : 工法種類
    # 'FrameInsulcolumn'（軸組構法）/'WallInsul'（枠組壁構法）
    
    if type == 'Floor' or type == 'BoundaryCeiling' or type == 'BoundaryFloor' :
        if ConstructionMethod == 'FrameInsulcolumn' :
            Ur = 0.13
        elif ConstructionMethod == 'WallInsul' :
            Ur = 0.08
        else :
            raise ValueError

    # WallInsulationMethod : 壁の断熱種類
    # 'InsulColumn'（充填断熱（柱・間柱間に断熱））/'InsulColumnandadd'（充填断熱（柱・間柱間に断熱）+付加断熱）/
    # 'OuterInsulMudWall'（土壁で外張断熱）/'OuterInsul'（土壁以外で外張断熱）    
    elif type == 'Wall' or type == 'BoundaryWall' :
        if WallInsulationMethod == 'InsulColumn' or RoofInsulationMethod == 'InsulColumnandadd' :
            if ConstructionMethod == 'FrameInsulcolumn' :
                Ur = 0.09
            elif ConstructionMethod == 'WallInsul' :
                Ur = 0.13
            else :
                raise ValueError
        elif WallInsulationMethod == 'OuterInsulMudWall' :
            Ur = 0.04
        elif WallInsulationMethod == 'OuterInsul' :
            Ur = 0.02
        else :
            raise ValueError 

    elif type == 'Ceiling' :
        Ur = 0.05

    # RoofInsulationMethod : 屋根の断熱種類
    # 'InsulRafter'（充填断熱（たるき間に断熱））/'InsulRafterandadd'（充填断熱（たるき間に断熱）+付加断熱）/'OuterInsul'（外張断熱工法）
    elif type == 'Roof' :
        if RoofInsulationMethod == 'InsulRafter' or RoofInsulationMethod == 'InsulRafterandadd' :
            Ur = 0.11
        elif RoofInsulationMethod == 'OuterInsul' :
            Ur = 0.02
        else :
            raise ValueError 

    else :
        raise ValueError
    
    return Ur

<div style="text-align: center;"> 表8 鉄骨造における一般部位の熱橋部分（柱及び梁以外）の仕様に応じた補正熱貫流率 </div>

| 「外装材＋断熱補強材」の熱抵抗(m<sup>2</sup>･K/W) | 補正熱貫流率ܷ |
|---|---|
| 1.7以上 | 0.00 |
| 1.7未満1.5以上 | 0.10 |
| 1.5未満1.3以上 | 0.13 |
| 1.3未満1.1以上 | 0.14 |
| 1.1未満0.9以上 | 0.18 |
| 0.9未満0.7以上 | 0.22 |
| 0.7未満0.5以上 | 0.40 |
| 0.5未満0.3以上 | 0.45 |
| 0.3未満0.1以上 | 0.60 |
| 0.1未満 | 0.70 |

In [16]:
def get_Ur_steel(Radding):
    # Rc : 「外装材＋断熱補強材」の熱抵抗, m2K/W
    
    if Radding >= 1.7 :
        Ur = 0.00
    elif Radding >= 1.5 :
        Ur = 0.10
    elif Radding >= 1.3 :
        Ur = 0.13    
    elif Radding >= 1.1 :
        Ur = 0.14    
    elif Radding >= 0.9 :
        Ur = 0.18    
    elif Radding >= 0.7 :
        Ur = 0.22    
    elif Radding >= 0.5 :
        Ur = 0.40    
    elif Radding >= 0.3 :
        Ur = 0.45    
    elif Radding >= 0.1 :
        Ur = 0.60  
    elif Radding < 0.1 :
        Ur = 0.70
    else :
        raise ValueError

    return Ur

$$
\displaystyle
U_{i}= \frac{1}{R_{g,i}} + U_{r,i}
$$

$U_{i}$: 一般部位$i$の熱貫流率, W/(m<sup>2</sup>・K)  
$R_{g,i}$: 一般部位$i$の断熱部分の熱抵抗, m<sup>2</sup>K/W  
$U_{r,i}$: 一般部位$i$の補正熱貫流率, W/(m<sup>2</sup>・K)  

In [17]:
def calc_UA_from_Ur(type, structure, IsInContactWithOutsideAir, Parts, ConstructionMethod = None, RoofInsulationMethod = None, 
                    WallInsulationMethod = None, Radding = None, **kwargs):
    
    # 断熱部分の熱抵抗, m2K/W
    Ro, Ri = get_Ro_Ri(type, IsInContactWithOutsideAir)
    Rg = Ro + sum(x['thick']/x['cond'] for x in Parts[0]['Layers']) + Ri
    
    # 補正熱貫流率
    if structure == 'wood' :
        Ur = get_Ur_wood(type, ConstructionMethod, RoofInsulationMethod, WallInsulationMethod)
    elif structure == 'steel' :
        Ur = get_Ur_steel(Radding)
    else :
        raise ValueError
    
    return 1/Rg + Ur

#### Example

In [18]:
d = {
    'Walls' : [
        {
            'area': 10,
            'direction' : 'Bottom',
            'IsInContactWithOutsideAir': True,
            'name' : 'Floor',
            'space' : 'Main',
            'nextspace' : 'OutsideAir',
            'structure' : 'wood', 
            'type' : 'Floor',
            'InputMethod' : 'InputUR',
            'ConstructionMethod' : 'WallInsul',
            'Parts' :  [{'Layers' : [{'cond': 0.16, 'name': 'PED', 'specH': 720.0, 'thick': 0.012}]}]
        },
        {
            'area': 20,
            'direction' : 'Top',
            'IsInContactWithOutsideAir': True,
            'name' : 'Roof',
            'space' : 'Main',
            'nextspace' : 'OutsideAir',
            'structure' : 'wood', 
            'type' : 'Roof',
            'InputMethod' : 'InputUR',
            'ConstructionMethod' : 'FrameInsulcolumn',
            'RoofInsulationMethod' : 'InsulRafterandadd',
            'Parts' :  [{'Layers' : [{'cond': 0.16, 'name': 'PED', 'specH': 720.0, 'thick': 0.012}]}]
        },
        {
            'area': 30,
            'direction' : 'North',
            'IsInContactWithOutsideAir': True,
            'name' : 'Wall',
            'space' : 'Main',
            'nextspace' : 'OutsideAir',
            'structure' : 'steel', 
            'type' : 'Wall',
            'InputMethod' : 'InputUR',
            'Radding' : 1.0,
            'Parts' :  [{'Layers' : [{'cond': 0.16, 'name': 'PED', 'specH': 720.0, 'thick': 0.012}]}]
        }
    ]
}

def get_UA_from_Ur(d):
    for x in d['Walls'] :
        if x['InputMethod'] == 'InputUR' :
            x['UA'] =  calc_UA_from_Ur(**x)
    return d

print(get_UA_from_Ur(d))

{'Walls': [{'area': 10, 'direction': 'Bottom', 'IsInContactWithOutsideAir': True, 'name': 'Floor', 'space': 'Main', 'nextspace': 'OutsideAir', 'structure': 'wood', 'type': 'Floor', 'InputMethod': 'InputUR', 'ConstructionMethod': 'WallInsul', 'Parts': [{'Layers': [{'cond': 0.16, 'name': 'PED', 'specH': 720.0, 'thick': 0.012}]}], 'UA': 3.853584905660377}, {'area': 20, 'direction': 'Top', 'IsInContactWithOutsideAir': True, 'name': 'Roof', 'space': 'Main', 'nextspace': 'OutsideAir', 'structure': 'wood', 'type': 'Roof', 'InputMethod': 'InputUR', 'ConstructionMethod': 'FrameInsulcolumn', 'RoofInsulationMethod': 'InsulRafterandadd', 'Parts': [{'Layers': [{'cond': 0.16, 'name': 'PED', 'specH': 720.0, 'thick': 0.012}]}], 'UA': 4.988048780487805}, {'area': 30, 'direction': 'North', 'IsInContactWithOutsideAir': True, 'name': 'Wall', 'space': 'Main', 'nextspace': 'OutsideAir', 'structure': 'steel', 'type': 'Wall', 'InputMethod': 'InputUR', 'Radding': 1.0, 'Parts': [{'Layers': [{'cond': 0.16, 'name

#### RC造

$$
\displaystyle
U_{i}= U_{i,k}
$$

$U_{i}$: 一般部位$i$の熱貫流率, W/(m<sup>2</sup>・K)  
$U_{i,k}$: 一般部位$i$の部分$k$の熱貫流率, W/(m<sup>2</sup>・K)  

In [19]:
def calc_UA_from_layer(type, IsInContactWithOutsideAir, Parts):
    
    # 断熱部分の熱抵抗, m2K/W
    Ro, Ri = get_Ro_Ri(type, IsInContactWithOutsideAir)
    Rg = Ro + sum(x['thick']/x['cond'] for x in Parts[0]['Layers']) + Ri
    
    return 1/Rg

#### 応答係数算出用層構成の作成

一般部位の熱貫流率$U_A$に一致するよう、一般部位の部分の層構成を調整する。  
－$U_A$に一致するまで、Layersの順番に厚を減らす  
－$U_A$に一致するまで、Layersの逆順に厚を減らす  
－$U_A$に一致するまで、熱伝導率の小さい層より厚を減らす（同じ熱伝導率の層が複数ある場合にはLayersのリストの順番）  
－$U_A$に一致するまで、熱伝導率の小さい層より厚を減らす（同じ熱伝導率の層が複数ある場合にはLayersのリストの逆順）  

In [20]:
# UAをもとに層構成を調整
# Layersの順番に厚を減らす
def get_layers_from_UA_acending(layers, UA, Ro, Ri):
    
    if UA < 1/(Ro + Ri + sum(x['thick']/x['cond'] for x in layers)) :
        raise ValueError
    elif UA > 1/(Ro + Ri) :
        raise ValueError
    else :
        for x in layers :
            if UA > 1/(Ro + Ri + sum(y['thick']/y['cond'] for y in layers)) :
                x['thick'] = max(0, (1/UA - Ro - Ri - (sum(y['thick']/y['cond'] for y in layers)-x['thick']/x['cond']))*x['cond'])
    
    return layers

In [21]:
# UAをもとに層構成を調整
# Layersの逆順に厚を減らす
def get_layers_from_UA_descending(layers, UA, Ro, Ri):
    
    if UA < 1/(Ro + Ri + sum(x['thick']/x['cond'] for x in layers)) :
        raise ValueError
    elif UA > 1/(Ro + Ri) :
        raise ValueError
    else :
        for x in reversed(layers) :
            if UA > 1/(Ro + Ri + sum(y['thick']/y['cond'] for y in layers)) :
                x['thick'] = max(0, (1/UA - Ro - Ri - (sum(y['thick']/y['cond'] for y in layers)-x['thick']/x['cond']))*x['cond'])
    
    return layers

In [22]:
# UAをもとに層構成を調整
# 熱伝導率の小さい層より厚を減らす
# 同じ熱伝導率の層が複数ある場合にはLayersのリストの順番に処理
def get_layers_from_UA_ins_acending(layers, UA, Ro, Ri):
    
    def get_cond_sort_list(l, c_list):
        # l : Layers
        
        l_copy = []
        for x in l :
            l_copy.append({'name': x['name'], 'cond': x['cond'], 'specH': x['specH'], 'thick': x['thick']})
        sort_list = []
        
        for x in range(0,len(l_copy)) :
            cond_temp = c_list[x]
            for y in range(0,len(l_copy)) :
                if cond_temp == l_copy[y]['cond'] :
                    sort_list.append(y)
                    l_copy[y]['cond'] = 0
                    cond_temp = -1                   
        return sort_list
    
    # Layers中の熱伝導率を昇順に並べる
    cond_list = sorted([x['cond'] for x in layers])

    # Layers中の熱伝導率の昇順の順番を取得する
    cond_sort_list = get_cond_sort_list(layers, cond_list)

    # 熱伝導率の小さい層より厚を調整
    # 同じ熱伝導率の層が複数ある場合にはLayersのリストの順番に処理
    if UA < 1/(Ro + Ri + sum(x['thick']/x['cond'] for x in layers)) :
        raise ValueError
    elif UA > 1/(Ro + Ri) :
        raise ValueError
    else :
        for x in cond_sort_list :
            if UA > 1/(Ro + Ri + sum(y['thick']/y['cond'] for y in layers)) :
                R = Ro + Ri + sum(y['thick']/y['cond'] for y in layers) - layers[x]['thick']/layers[x]['cond']
                layers[x]['thick'] = max(0, (1/UA - R)*layers[x]['cond'])
    
    return layers

In [23]:
# UAをもとに層構成を調整
# 熱伝導率の小さい層より厚を減らす
# 同じ熱伝導率の層が複数ある場合にはLayersのリストの逆順に処理
def get_layers_from_UA_ins_descending(layers, UA, Ro, Ri):
    
    def get_cond_sort_list(l, c_list):
        # l : Layers
        
        l_copy = []
        for x in l :
            l_copy.append({'name': x['name'], 'cond': x['cond'], 'specH': x['specH'], 'thick': x['thick']})
        sort_list = []
        
        sort_list = []
        for x in range(0,len(l_copy)) :
            cond_temp = c_list[x]
            for y in range(len(l_copy),0,-1) :
                if cond_temp == l_copy[y-1]['cond'] :
                    sort_list.append(y-1)
                    l_copy[y-1]['cond'] = 0
                    cond_temp = -1                   
        return sort_list
    
    # Layers中の熱伝導率を昇順に並べる
    cond_list = sorted([x['cond'] for x in layers])

    # Layers中の熱伝導率の昇順の順番を取得する
    cond_sort_list = get_cond_sort_list(layers, cond_list)

    # 熱伝導率の小さい層より厚を調整
    # 同じ熱伝導率の層が複数ある場合にはLayersのリストの逆順に処理
    if UA < 1/(Ro + Ri + sum(x['thick']/x['cond'] for x in layers)) :
        raise ValueError
    elif UA > 1/(Ro + Ri) :
        raise ValueError
    else :
        for x in cond_sort_list :
            if UA > 1/(Ro + Ri + sum(y['thick']/y['cond'] for y in layers)) :
                R = Ro + Ri + sum(y['thick']/y['cond'] for y in layers) - layers[x]['thick']/layers[x]['cond']
                layers[x]['thick'] = max(0, (1/UA - R)*layers[x]['cond'])
    
    return layers

#### 応答係数算出用の層構成の決定

応答係数算出用の層構成を決定する。  
－簡略計算法①の断熱部分  
－面積比率が最大の部分  

In [24]:
def get_part_gneral_wood_floor(FloorConstructionMethod):
    # FloorConstructionMethod: 床の工法種類
    # 'FrameBeamInsuljoist'（軸組構法・床梁工法（根太間に断熱））/'FrameFootingInsuljoist'（軸組構法・束立大引工法（根太間に断熱））/
    # 'FrameFootingInsulsleeper'（軸組構法・束立大引工法（大引間に断熱））/
    # 'FrameFootingInsuljoistSleeper'（軸組構法・束立大引工法（根太間及び大引間に断熱））/
    # 'FrameRigidfloor'（軸組構法・剛床工法）/'FrameBeambaseInsuljoist'（軸組構法・床梁土台同面工法（根太間に断熱））/
    # 'WallInsuljoist'（枠組壁工法（根太間に断熱））
    
    if FloorConstructionMethod == 'FrameBeamInsuljoist' :
        part_gneral = 'Insulation'
    elif FloorConstructionMethod == 'FrameFootingInsuljoist' :
        part_gneral = 'Insulation'
    elif FloorConstructionMethod == 'FrameFootingInsulsleeper' :
        part_gneral = 'Insulation'
    elif FloorConstructionMethod == 'FrameFootingInsuljoistSleeper' :
        part_gneral = 'InsulJoitandsleeper'
    elif FloorConstructionMethod == 'FrameRigidfloor' :
        part_gneral = 'Insulation'
    elif FloorConstructionMethod == 'FrameBeambaseInsuljoist' :
        part_gneral = 'Insulation'
    elif FloorConstructionMethod == 'WallInsuljoist' :
        part_gneral = 'Insulation'
    else :
        raise ValueError
    
    return part_gneral

In [25]:
def get_part_gneral_wood_wall(WallConstructionMethod):
    # WallConstructionMethod: 床の工法種類
    # 'FrameInsulcolumn'（軸組構法・柱、間柱間に断熱）/
    # 'FrameInsuladdBackhorizontal'（軸組構法・柱、間柱間に断熱し付加断熱（付加断熱層内熱橋部分が「横下地」））/
    # 'FrameInsuladdBackvertical'（軸組構法・柱、間柱間に断熱し付加断熱（付加断熱層内熱橋部分が「縦下地」））/
    # 'WallInsuljamb'（枠組壁工法・たて枠間に断熱）/
    # 'WallInsuladdBackhorizontal'（枠組壁工法・たて枠間に断熱＋付加断熱（付加断熱層内熱橋部分が「横下地」））/
    # 'WallInsuladdBackvertical'（枠組壁工法・たて枠間に断熱＋付加断熱（付加断熱層内熱橋部分が「縦下地」））
    
    if WallConstructionMethod == 'FrameInsulcolumn' :
        part_gneral = 'Insulation'
    elif WallConstructionMethod == 'FrameInsuladdBackhorizontal' :
        part_gneral = 'FrameInsulColumnandadd'
    elif WallConstructionMethod == 'FrameInsuladdBackvertical' :
        part_gneral = 'FrameInsulColumnandadd'
    elif WallConstructionMethod == 'WallInsuljamb' :
        part_gneral = 'Insulation'
    elif WallConstructionMethod == 'WallInsuladdBackhorizontal' :
        part_gneral = 'WallInsulFilledandadd'
    elif WallConstructionMethod == 'WallInsuladdBackvertical' :
        part_gneral = 'WallInsulFilledandadd'
    else :
        raise ValueError
    
    return part_gneral

In [26]:
def get_part_gneral_wood_ceiling(CeilingConstructionMethod):
    # CeilingConstructionMethod: 天井の工法種類
    # 'Insulbeam'（桁・梁間に断熱）
    
    if CeilingConstructionMethod == 'Insulbeam' :
        part_gneral = 'Insulation'
    else :
        raise ValueError

    return part_gneral

In [27]:
def get_part_gneral_wood_roof(RoofConstructionMethod):
    # RoofConstructionMethod : 屋根の工法種類
    # 'Insulrafter'（たるき間に断熱）/'InsuladdBackhorizontal'（たるき間に断熱＋付加断熱（横下地）））
    
    if RoofConstructionMethod == 'Insulrafter' :
        part_gneral = 'Insulation'
    elif RoofConstructionMethod == 'InsuladdBackhorizontal' :
        part_gneral = 'InsulRafterandadd'
    else :
        raise ValueError
    
    return part_gneral

#### 応答係数算出

層構成より応答係数を算出する。  

In [28]:
def get_responsce_factor_from_layers(name, Ro, Ri, Layers) :

    import nbimporter
    import heatload_calc.Wall as mf
    
    layers =[mf.Layer( x['name'], x['cond'], x['specH'], x['thick'] ) for x in Layers]
    layers.append(mf.Layer( 'OutHeatTrans', 1/Ro, 0.0, 0.00 ))

    hr = 5.0 
    wall = mf.Wall(name, 0.90, 0.80, 1/Ri - hr, hr, layers)

    rf =mf.ResponseFactor('wall', 3600, 50, wall)

    return {'RFT0' : rf.RFT0(), 'RFT1' : rf.RFT1() ,'RFA0' : rf.RFA0(), 'RFA1' : rf.RFA1()}

#### 一般部位構造種別・壁体仕様入力方法(木造)に応じた応答係数の算出

一般部位構造種別・壁体仕様入力方法(木造)に応じて、一般部位のU値の算出、応答係数算出用の層構成の決定、応答係数の算出を行う。  

In [29]:
def copy_layers(Layers) :
    
    l = []
    for x in Layers :
        l.extend([ { 'name' : x['name'], 'cond' : x['cond'], 'thick' : x['thick'], 'specH' : x['specH'] }])

    return l

In [30]:
def get_responce_factor(IsSimplifiedInput, name, type, structure, area, direction, IsInContactWithOutsideAir, UA = None, Parts = None, 
                        InputMethod = None, RoofConstructionMethod = None, CeilingConstructionMethod = None, WallConstructionMethod = None, 
                        FloorConstructionMethod = None, ConstructionMethod = None, RoofInsulationMethod = None, 
                        WallInsulationMethod = None, Radding = None, **kwargs):  
    
    Parts_for_RF = []
    
    # 表面熱伝達抵抗の取得
    Ro, Ri = get_Ro_Ri(type, IsInContactWithOutsideAir)    
    
    # 簡易外皮入力
    if IsSimplifiedInput == True or InputMethod == 'InputUA' :
        #層構成の取得
        import nbimporter
        import ConvertUValueToSpec as mf       
        layers = mf.convert_u_value_to_spec( structure, type, UA )
        Parts_for_RF = [
                    {'area': area,
                     'Layers' : [ { 'name' : l.name, 'cond' : l.cond, 'thick' : l.thick, 'specH' : l.spech } for l in layers ]}
        ]
        
    # 木造 詳細計算法
    elif structure == 'wood' and InputMethod == 'InputAllDetails' :
        for x in Parts :
            Parts_for_RF.append({'area':x['AreaRatio']*area, 'Layers': copy_layers(x['Layers'])})
                
    # 木造 簡略計算法①
    elif (structure == 'wood' and InputMethod == 'InputAllLayers') :
        area_ratio = get_area_ratio_wood(type, RoofConstructionMethod, CeilingConstructionMethod, 
                                         WallConstructionMethod, FloorConstructionMethod)
        for x in Parts :
            # 面積の取得
            if type == 'Roof' or type == 'Ceiling' :
                area_part = area_ratio[x['TypeRoof']]*area   
            elif type == 'BoundaryCeiling' :
                area_part =  area_ratio[x['TypeCeiling']]*area 
            elif type == 'Wall' or type == 'BoundaryWall' :
                area_part = area_ratio[x['TypeWall']]*area 
            elif type == 'Floor' or type == 'BoundaryFloor' :
                area_part = area_ratio[x['TypeFloor']]*area 
           # Part_for_RFへの追加    
            Parts_for_RF.append({'area':area_part, 'Layers': copy_layers(x['Layers'])})
                           
    # 木造 簡略計算法②、鉄骨造
    elif (structure == 'wood' and InputMethod == 'InputUR') \
      or (structure == 'steel' and InputMethod == 'InputUR') :
        UA = calc_UA_from_Ur(type, structure, IsInContactWithOutsideAir, Parts,   
                             ConstructionMethod, RoofInsulationMethod, WallInsulationMethod, Radding)
        Parts_for_RF.append({'area':area, 'Layers': get_layers_from_UA_ins_descending(Parts[0]['Layers'], UA, Ro, Ri)})
    
    # RC造
    elif (structure == 'RC' and InputMethod == 'InputLayers') :
        UA = calc_UA_from_layer(type, IsInContactWithOutsideAir, Parts)
        Parts_for_RF.append({'area':area, 'Layers': get_layers_from_UA_ins_descending(Parts[0]['Layers'], UA, Ro, Ri)})
        
    # その他
    elif structure == 'other' :
        raise ValueError
        
    else :
        raise ValueError
  

    # 応答係数の取得
    for x in Parts_for_RF :
        x['ResponceFactor'] = get_responsce_factor_from_layers(name, Ro, Ri, x['Layers'])
        x.pop('Layers')
        
    return Parts_for_RF

In [31]:
d = {
    'Common': {'Region': 6,
               'MainOccupantRoomFloorArea': 30.0,
               'OtherOccupantRoomFloorArea': 60.0,
               'TotalFloorArea': 120.0,
               'IsSimplifiedInput': False
              },    
    'Walls' : [
        {
            'area' : 20,
            'direction' : 'Bottom',
            'IsInContactWithOutsideAir' : True,
            'name' : 'Floor',
            'space' : 'Main',
            'nextspace' : 'ClosedBackFloor',
            'structure' : 'wood', 
            'type' : 'Floor',
            'UA': 1.0,
            'InputMethod' : 'InputUA',
        },
        {
            'area' : 20,
            'direction' : 'Bottom',
            'IsInContactWithOutsideAir': True,
            'name' : 'Floor',
            'space' : 'Main',
            'nextspace' : 'ClosedBackFloor',
            'structure' : 'wood', 
            'type' : 'Floor',
            'UA': 1.0,
            'InputMethod' : 'InputAllDetails',
            'Parts' :  [
                {'AreaRatio': 0.5, 'Layers' : [{'cond': 0.16, 'name': 'PED', 'specH': 720.0, 'thick': 0.024}]},
                {'AreaRatio': 0.5, 'Layers' : [{'cond': 0.16, 'name': 'PED', 'specH': 720.0, 'thick': 0.012}]}
            ]
         },
        {
            'area' : 20,
            'direction' : 'Top',
            'IsInContactWithOutsideAir': True,
            'name' : 'Roof',
            'space' : 'Main',
            'nextspace' : 'OutsideAir',
            'structure' : 'wood', 
            'type' : 'Roof',
            'InputMethod' : 'InputAllLayers',
            'RoofConstructionMethod' : 'InsuladdBackhorizontal',
            'Parts' :  [
                {'TypeRoof': 'InsulRafterandadd', 'Layers' : [{'cond': 0.16, 'name': 'PED', 'specH': 720.0, 'thick': 0.012}]},
                {'TypeRoof': 'InsulrafterHeatbridgeadd', 'Layers' : [{'cond': 0.16, 'name': 'PED', 'specH': 720.0, 'thick': 0.012}]},
                {'TypeRoof': 'InsuladdHeatbridgemember', 'Layers' : [{'cond': 0.16, 'name': 'PED', 'specH': 720.0, 'thick': 0.012}]},
                {'TypeRoof': 'HeatbridgeMemberandadd', 'Layers' : [{'cond': 0.16, 'name': 'PED', 'specH': 720.0, 'thick': 0.012}]}
            ]
        },
        {
            'area' : 30,
            'direction' : 'North',
            'IsInContactWithOutsideAir': True,
            'name' : 'Wall',
            'space' : 'Main',
            'structure' : 'wood', 
            'nextspace' : 'OutsideAir',
            'type' : 'Wall',
            'InputMethod' : 'InputAllLayers',
            'WallConstructionMethod' : 'WallInsuladdBackvertical',
            'Parts' :  [
                {'TypeWall': 'WallInsulFilledandadd', 'Layers': [{'cond': 0.16, 'name': 'PED', 'specH': 720.0, 'thick': 0.012}]},
                {'TypeWall': 'WallInsulfilledHeatbridgeadd', 'Layers': [{'cond': 0.16, 'name': 'PED', 'specH': 720.0, 'thick': 0.012}]},
                {'TypeWall': 'WallInsuladdHeatbridgemember', 'Layers': [{'cond': 0.16, 'name': 'PED', 'specH': 720.0, 'thick': 0.012}]},
                {'TypeWall': 'WallInsuladdHeatbridgelintel', 'Layers': [{'cond': 0.16, 'name': 'PED', 'specH': 720.0, 'thick': 0.012}]},
                {'TypeWall': 'WallHeatbridgeMemberandadd', 'Layers': [{'cond': 0.16, 'name': 'PED', 'specH': 720.0, 'thick': 0.012}]},
                {'TypeWall': 'WallHeatbridgeLintelandadd', 'Layers': [{'cond': 0.16, 'name': 'PED', 'specH': 720.0, 'thick': 0.012}]}
            ]
        }
    ]
}

d_rf = []
for x in d['Walls'] :
    d_rf.extend([get_responce_factor(d['Common']['IsSimplifiedInput'], **x)])
    
print(d_rf)

Importing Jupyter notebook from ConvertUValueToSpec.ipynb
Importing Jupyter notebook from C:\Users\serikawa\Documents\e7_inputdataconvert\heatload_calc\Wall.ipynb
[[{'area': 20, 'ResponceFactor': {'RFT0': 0.75363770902550153, 'RFT1': [2.6122690155725085e-05, -0.00044668380212315747, 0.0040839037621566571, -0.02528270738695293, 0.087986915852958647, 0.21199844140314972, -0.029791460364175435, 0.0019362314738760619], 'RFA0': 0.79962523826120857, 'RFA1': [1.5933799254523897e-06, -2.7867261448198028e-05, 0.00026698944792924467, -0.0018138875131411909, 0.0075453179009085259, 0.024454892504155754, 0.017353106604788063, 0.0010707139518562716]}}], [{'area': 10.0, 'ResponceFactor': {'RFT0': 0.50888151048998476, 'RFT1': [3.5831330163207947e-05, -0.00062442253685340702, 0.0059836660602139895, -0.042700632325610229, 0.46376050084383735, 0.021449282028832749, -0.01243488617146218, 0.001229243134991554], 'RFA0': 0.1253697277315324, 'RFA1': [3.5158643874704846e-06, -6.1881648124077992e-05, 0.00060619

#### 応答係数の係数の集約

簡易熱負荷計算でひとつの部位とみなされる部位・部分の応答係数を集約する。    
どの部位・部分を集約するかは既知であるとする。ここでは、暫定的に、同じ用途、同じ方位の部位を集約するものとする。

In [32]:
# 面積加重平均
def get_area_weighted_average(area_old, area_add, value_old, value_add):
        return (area_old*value_old + area_add*value_add)/(area_old + area_add)

In [33]:
def  copy_responce_factor(RF):
    
    RF_copy = {
        'RFA0': RF['RFA0'],
        'RFA1': [x for x in RF['RFA1']],
        'RFT0': RF['RFT0'],
        'RFT1': [x for x in RF['RFT1']]
    }
    
    return RF_copy

In [34]:
def  integrate_responce_factor(d):

    d_responce_factor = {}
    for x in d :
        for y in x['Parts_RF'] :
            name = x['space']+'_'+x['direction']+'_'+x['type']
            
            # 他の部位・部分の応答係数に集約
            if (name in d_responce_factor) == True :       

                area_old = d_responce_factor[name]['area']
                area_add = y['area']

                responce_factor_old = copy_responce_factor(d_responce_factor[name]['ResponceFactor'])
                responce_factor_add = y['ResponceFactor']

                d_responce_factor[name]['area'] = area_old + area_add

                # 応答係数の面積加重平均値の取得
                for z in ['RFA', 'RFT']:
                    d_responce_factor[name]['ResponceFactor'][z+'0'] \
                        = get_area_weighted_average(area_old, area_add, \
                                                    responce_factor_old[z+'0'],responce_factor_add[z+'0'])
                    for y in range(0,len(responce_factor_old[z+'1'])) :
                        d_responce_factor[name]['ResponceFactor'][z+'1'][y] \
                            = get_area_weighted_average(area_old, area_add, \
                                                    responce_factor_old[z+'1'][y],responce_factor_add[z+'1'][y])
            
            # 当該部位の応答係数を使用
            else :
                d_responce_factor[name] = {'area':y['area'], 'direction':x['direction'], 'type':x['type'], 
                                           'ResponceFactor':y['ResponceFactor']}
    
    # 辞書からリストへ
    l_responce_factor = []
    for x in d_responce_factor.keys() :
        l_responce_factor.append(d_responce_factor[x])
                
    return  l_responce_factor

#### Example

In [35]:
d = {
    'Walls' : [
        {
            'area': 20,
            'direction' : 'Bottom',
            'name' : 'Floor',
            'space' : 'main',
            'type' : 'Floor',
            'Parts_RF' :  [
                {'area': 20,
                 'ResponceFactor': {
                     'RFT0': 0.75363770902550153,
                     'RFT1': [2.6122690155725085e-05,  -0.00044668380212315747, 0.0040839037621566571, -0.02528270738695293,
                              0.087986915852958647, 0.21199844140314972, -0.029791460364175435,  0.0019362314738760619], 
                     'RFA0': 0.79962523826120857, 
                     'RFA1': [1.5933799254523897e-06, -2.7867261448198028e-05, 0.00026698944792924467, -0.0018138875131411909,
                              0.0075453179009085259, 0.024454892504155754, 0.017353106604788063, 0.0010707139518562716]}}, 
                 {'area': 10.0, 
                  'ResponceFactor': {
                      'RFT0': 0.50888151048998476, 
                      'RFT1': [3.5831330163207947e-05, -0.00062442253685340702, 0.0059836660602139895, -0.042700632325610229,
                               0.46376050084383735, 0.021449282028832749, -0.01243488617146218, 0.001229243134991554],
                      'RFA0': 0.1253697277315324, 
                      'RFA1': [3.5158643874704846e-06, -6.1881648124077992e-05, 0.00060619324351350698, -0.0045492960707860493,
                               0.056465597921361327, 0.0047485894532887955, 0.00013265253042132798, 0.00014893084345409853]}}
                 ]
         },
        {
            'area': 20,
            'direction' : 'Top',
            'name' : 'Roof',
            'space' : 'main',
            'structure' : 'wood', 
            'type' : 'Ceiling',
            'Parts_RF' :  [
                 {'area': 10.0, 
                  'ResponceFactor': {
                      'RFT0': 0.50888151048998476, 
                      'RFT1': [3.5831330163207947e-05, -0.00062442253685340702, 0.0059836660602139895, -0.042700632325610229,
                               0.46376050084383735, 0.021449282028832749, -0.01243488617146218, 0.001229243134991554],
                      'RFA0': 0.1253697277315324, 
                      'RFA1': [3.5158643874704846e-06, -6.1881648124077992e-05, 0.00060619324351350698, -0.0045492960707860493,
                               0.056465597921361327, 0.0047485894532887955, 0.00013265253042132798, 0.00014893084345409853]}}
            ]
        }    ]
}
    
print(integrate_responce_factor(d['Walls']))

[{'area': 30.0, 'direction': 'Bottom', 'type': 'Floor', 'ResponceFactor': {'RFT0': 0.6720523095136627, 'RFT1': [2.9358903491552707e-05, -0.0005059300470332406, 0.004717157861509101, -0.031088682366505364, 0.21324477751658488, 0.14848205494504407, -0.024005935633271018, 0.0017005686942478926], 'RFA0': 0.5748734014179833, 'RFA1': [2.2342080794584213e-06, -3.920539034015802e-05, 0.00038005737979066544, -0.0027256903656894768, 0.023852077907726126, 0.017886124820533433, 0.011612955246665817, 0.000763452915722214]}}, {'area': 10.0, 'direction': 'Top', 'type': 'Ceiling', 'ResponceFactor': {'RFT0': 0.5088815104899848, 'RFT1': [3.583133016320795e-05, -0.000624422536853407, 0.0059836660602139895, -0.04270063232561023, 0.46376050084383735, 0.02144928202883275, -0.01243488617146218, 0.001229243134991554], 'RFA0': 0.1253697277315324, 'RFA1': [3.5158643874704846e-06, -6.188164812407799e-05, 0.000606193243513507, -0.004549296070786049, 0.05646559792136133, 0.0047485894532887955, 0.000132652530421327

## 2. Convert

### 2.1 Common Element

#### Outline

'Common'要素はそのまま維持する。

#### Function

In [36]:
def convert_common(d):
    return {
            'Region'                     : d['Common']['Region'],
            'IsSimplifiedInput'          : d['Common']['IsSimplifiedInput'],
            'MainOccupantRoomFloorArea'  : d['Common']['MainOccupantRoomFloorArea'],
            'OtherOccupantRoomFloorArea' : d['Common']['OtherOccupantRoomFloorArea'],
            'TotalFloorArea'             : d['Common']['TotalFloorArea']
    }

### 2.2 Walls Element

#### Outline

'Walls'要素に床フラグを追加する。それ以外はそのまま維持する。

#### Function

In [37]:
def make_layer(name, thick, cond, specH, **kwargs) :
    
    return {
            'name'  : name,
            'thick' : thick,
            'cond'  : cond,
            'specH' : specH
    }

In [38]:
def make_parts(InputMethod, Parts, type = None, **kwargs) :
    
    parts_copy = []
    
    for x in Parts :
        part = {}
        if InputMethod == 'InputAllDetails' :
            part['AreaRatio'] = x['AreaRatio']
        elif InputMethod == 'InputAllLayers' :
            if type == 'Ceiling' :
                part['TypeRoof'] = x['TypeRoof']
            elif type == 'Wall' or type == 'BoundaryWall' :
                part['TypeWall'] = x['TypeWall']
            elif type == 'Floor' or type == 'BoundaryFloor' :
                part['TypeFloor'] = x['TypeFloor']
            elif type == 'BoundaryCeiling' :
                part['TypeCeiling'] = x['TypeCeiling']
            else :
                raise ValueError       
            
        part['Layers'] = [make_layer(**layer) for layer in x['Layers']]
        parts_copy.append(part)
        
    return parts_copy

In [39]:
def make_wall(IsSimplifiedInput, name, direction, IsInContactWithOutsideAir, area, space, type, IsSunshadeInput, Parts = None, 
              structure = None, nextspace = None, UA = None, InputMethod = None, UR = None, RoofConstructionMethod = None, 
              WallConstructionMethod = None, FloorConstructionMethod = None, CeilingConstructionMethod = None, 
              TypeFloor = None, TypeWall = None, TypeCeiling = None,TypeRoof = None, 
              ConstructionMethod = None, RoofInsulationMethod = None, WallInsulationMethod = None, Radding = None,
              Y1 = None, Y2 = None, Z = None, **kwargs ):
    
    wall = {
        'name':      name,
        'direction': direction,
        'space':     space,
        'type':      type,
        'IsSunshadeInput': IsSunshadeInput,
        'IsFloor': set_floor_flag(direction),
        'Parts_RF': get_responce_factor(IsSimplifiedInput, name, type, structure, area, direction, IsInContactWithOutsideAir, UA, Parts,
                                        InputMethod, RoofConstructionMethod, CeilingConstructionMethod, WallConstructionMethod, 
                                        FloorConstructionMethod, ConstructionMethod, RoofInsulationMethod, WallInsulationMethod, Radding)
    }
    
    # 日除けの設定
    if IsSunshadeInput == True :
        wall['Y1'] = Y1
        wall['Y2'] = Y2
        wall['Z'] = Z
 
    return wall

In [40]:
def convert_wall( d ):    
    
    d_wall = [ make_wall(IsSimplifiedInput = d['Common']['IsSimplifiedInput'], **f) for f in d['Walls'] ]   
    
    d_wall_integrate = integrate_responce_factor(d_wall)
    
    for x in d_wall_integrate :
        x['IsFloor'] = set_floor_flag(x['direction'])
    
    return d_wall_integrate

### 2.3 Windows Element

#### Outline

'Windows'要素に床フラグを追加する。それ以外はそのまま維持する。

#### Function

In [41]:
def make_window(IsSimplifiedInput, name, direction, area, space, IsSunshadeInput, UW, TypeWindow = None, IsEtaValueInput = None,
                EtaCooling = None, EtaHeating = None, Eta = None,TypeFrame = None, TypeGlass = None, TypeShade = None, 
                EtaInside = None,TypeFrameInside = None, TypeGlassInside = None, TypeShadeInside = None, 
                EtaOutside = None,TypeFrameOutside = None, TypeGlassOutside = None, TypeShadeOutside = None, 
                Y1 = None, Y2 = None, Z = None, **kwargs ):
    
    window = {
        'name':      name,
        'direction': direction,
        'area':      area,
        'space':     space,
        'UW':         UW,
        'IsSunshadeInput': IsSunshadeInput,
        'IsFloor': set_floor_flag(direction)
    } 
    
    # 簡易外皮入力の場合
    if IsSimplifiedInput == True :
            window['EtaCooling'] = EtaCooling 
            window['EtaHeating'] = EtaHeating    
            
    # 簡易外皮入力でない場合
    else :
        window['TypeWindow'] = TypeWindow
        window['IsEtaValueInput'] = IsEtaValueInput
        # 一重窓
        if TypeWindow == 'Single' :
            if IsEtaValueInput == True :
                window['Eta'] = Eta
            else :
                window['TypeFrame'] = TypeFrame
                window['TypeGlass'] = TypeGlass
                window['TypeShade'] = TypeShade
        # 二重窓
        elif TypeWindow == 'Double' :
            if IsEtaValueInput == True :
                window['EtaInside'] = EtaInside
                window['EtaOutside'] = EtaOutside
            else :
                window['TypeFrameInside'] = TypeFrameInside
                window['TypeGlassInside'] = TypeGlassInside
                window['TypeShadeInside'] = TypeShadeInside
                window['TypeFrameOutside'] = TypeFrameOutside
                window['TypeGlassOutside'] = TypeGlassOutside
                window['TypeShadeOutside'] = TypeShadeOutside
        else :
            raise ValueError        
    
    # 日除けの設定
    if IsSunshadeInput == True :
        # 一重窓
        if TypeWindow == 'Single' :
                window['TypeGlass'] = TypeGlass
        # 二重窓
        elif TypeWindow == 'Double' :
                window['TypeGlassInside'] = TypeGlassInside
                window['TypeGlassOutside'] = TypeGlassOutside
        else :
            raise ValueError        
                
        window['Y1'] = Y1
        window['Y2'] = Y2
        window['Z'] = Z

    return window

In [42]:
def convert_window( d ):    
    return [ make_window(IsSimplifiedInput = d['Common']['IsSimplifiedInput'], **f) for f in d['Windows'] ]

### 2.4 Doors Element

#### Outline

'Doors'要素に床フラグを追加する。それ以外はそのまま維持する。

#### Function

In [43]:
def make_door(name, direction, area, space, U, IsSunshadeInput, Y1 = None, Y2 = None, Z = None, **kwargs ):

    door = {
        'name':      name,
        'direction': direction,
        'area':      area,
        'space':     space,
        'U':         U,
        'IsSunshadeInput': IsSunshadeInput,
        'IsFloor': set_floor_flag(direction)
    }
    
    # 日除けの設定
    if IsSunshadeInput == True :
        door['Y1'] = Y1
        door['Y2'] = Y2
        door['Z'] = Z
    
    return door

In [44]:
def convert_door( d ):    
    return [ make_door(**f) for f in d['Doors'] ]

### 2.5 Heatbridges Element

#### Outline

'Heatbridges'要素はそのまま維持する。

#### Function

In [45]:
def make_heatbridge(name, structure, direction1, direction2, space, length, psi):
    return {
        'name':       name,
        'structure':  structure,
        'direction1': direction1,
        'direction2': direction2,
        'length':     length,
        'space':      space,
        'psi':        psi
    }

In [46]:
def convert_heatbridge( d ):    
    return [ make_heatbridge(**f) for f in d['Heatbridges'] ]

### 2.6 EarthfloorPerimeters Element

#### Outline

'EarthfloorPerimeters'要素はそのまま維持する。

#### Function

In [47]:
def make_earthfloorperimeter( direction, length, name, psi, space):
    return {
        'direction': direction,
        'length':    length,
        'name':      name,
        'psi':       psi,
        'space':     space
    }

In [48]:
def convert_earthfloorperimeter( d ):    
    return [ make_earthfloorperimeter(**f) for f in d['EarthfloorPerimeters'] ]

### 2.7 Earthfloors Element

#### Outline

'Earthfloors'要素に床フラグ：Trueを追加する。それ以外はそのまま維持する。

#### Function

In [49]:
def make_earthfloor( name, area, space ):
    return {
        'name': name,
        'area': area,
        'space': space,
        'IsFloor': True
    }

In [50]:
def convert_earthfloor( d ):        
    return [ make_earthfloor(**f) for f in d['Earthfloors'] ]

### 2.8 InnerWalls Element

#### Outline

'InnerWalls'要素に床フラグを追加する。層構成情報は応答係数に置き換える。それ以外はそのまま維持する。  
床下空間上の床、室上の床は、床フラグ：Trueとする。それ以外は、床フラグ：Falseとする。

#### Function

In [51]:
def set_floor_flag_innerwall(type):
    return True if type == 'GroundFloor' or type == 'InnerFloor' else False 

In [52]:
def make_innerwall(name, area, direction, type, Layers, spaces = None, space = None, nextspace = None, **kwargs):
    
    Ro, Ri = get_Ro_Ri_innerwall(type)
    
    if type == 'GroundFloor' or type == 'InnerWall' :
        return {
            'name': name,
            'area': area,
            'direction': direction,
            'spaces': spaces,
            'type' : type,
            'IsFloor': set_floor_flag_innerwall(type),
            'ResponceFactor': get_responsce_factor_from_layers(name, Ro, Ri, Layers)
        }
    
    elif type == 'InnerFloor' or type == 'InnerCeiling' :
        return {
            'name': name,
            'area': area,
            'direction': direction,
            'space': space,
            'nextspace': nextspace,
            'type' : type,
            'IsFloor': set_floor_flag_innerwall(type),
            'ResponceFactor': get_responsce_factor_from_layers(name, Ro, Ri, Layers)
        }

In [53]:
def convert_innerwall( d ):        
    return [ make_innerwall(**f) for f in d['InnerWalls'] ]

#### Example

In [54]:
d = {
    'InnerWalls': [
        {'area': 6.666666666666667,'direction': 'Horizontal', 'Layers': [{'cond': 0.16, 'name': 'PED', 'specH': 720.0, 'thick': 0.012}],
         'name': 'GroundFloor_other','spaces': ['other', 'underfloor'],'type': 'GroundFloor'},
        {'area': 53.333333333333336, 'direction': 'Horizontal', 'Layers': [{'cond': 0.16, 'name': 'PED', 'specH': 720.0, 'thick': 0.012}],
         'name': 'InnerFloor_other', 'nextspace': 'nonliving', 'space': 'other', 'type': 'InnerFloor'},
        {'area': 60.0, 'direction': 'Horizontal', 'Layers': [{'cond': 0.16, 'name': 'PED', 'specH': 720.0, 'thick': 0.012}],
         'name': 'InnerCeiling_other', 'nextspace': 'nonliving', 'space': 'other', 'type': 'InnerCeiling'},
        {'area': 121.45675773706458, 'direction': 'Vertical',
         'Layers': [{'cond': 0.22, 'name': 'GPB', 'specH': 830.0, 'thick': 0.0125},
                    {'cond': 11.11111111111111, 'name': 'GPB', 'specH': 0, 'thick': 1},
                    {'cond': 0.22, 'name': 'GPB', 'specH': 830.0, 'thick': 0.0125}],
         'name': 'Innerwall_other to nonliving', 'spaces': ['other', 'nonliving'], 'type': 'InnerWall'}]
}

convert_innerwall( d )

[{'IsFloor': True,
  'ResponceFactor': {'RFA0': 0.12515318851186069,
   'RFA1': [6.2485463820495407e-06,
    -0.00010811168759887066,
    0.0010160225134560163,
    -0.0067607967416226551,
    0.032019439170313004,
    0.014200860440099971,
    -0.0010707219674903046,
    0.00026338396414694718],
   'RFT0': 0.69664891123029715,
   'RFT1': [4.5327014252306456e-05,
    -0.00078576241201922436,
    0.0074156926107328122,
    -0.049803880466809225,
    0.24112326775540732,
    0.1139803357170411,
    -0.011972564386839668,
    0.0004802103606430642]},
  'area': 6.666666666666667,
  'direction': 'Horizontal',
  'name': 'GroundFloor_other',
  'spaces': ['other', 'underfloor'],
  'type': 'GroundFloor'},
 {'IsFloor': True,
  'ResponceFactor': {'RFA0': 0.12515318851186069,
   'RFA1': [6.2485463820495407e-06,
    -0.00010811168759887066,
    0.0010160225134560163,
    -0.0067607967416226551,
    0.032019439170313004,
    0.014200860440099971,
    -0.0010707219674903046,
    0.0002633839641469471

## 3.Lv5入力情報のコンバート（統合）

- 1)共通情報、2)開口部以外の不透明部位、3)開口部の透明部位、4)開口部の不透明部位、5) 線熱橋、6)土間床等の外周部の線熱橋、7)土間床、8)内壁等の各コンバート関数を統合し、Lv4の入力情報をLv5の入力情報にコンバートする。

### Function

In [55]:
def convert(d):
    d_lv5_input = {}
    d_lv5_input['Common'] = convert_common(d)
    if ('Walls' in d) == True:
        d_lv5_input['Walls'] = convert_wall(d)
    if ('Windows' in d) == True:
        d_lv5_input['Windows'] = convert_window(d)
    if ('Doors' in d) == True:
        d_lv5_input['Doors'] = convert_door(d)
    if ('EarthfloorPerimeters' in d) == True:
        d_lv5_input['EarthfloorPerimeters'] = convert_earthfloorperimeter(d)
    if ('EarthFloors' in d) == True:
        d_lv5_input['EarthFloors'] = convert_earthfloor(d)
    if ('Heatbridges' in d) == True:
        d_lv5_input['Heatbridges'] = convert_heatbridge(d)
    d_lv5_input['InnerWalls'] = convert_innerwall(d)

    return d_lv5_input

### Example

In [56]:
d = {
    'Common': {
        'Region': 6,
        'IsSimplifiedInput': False,
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0
    },
    'Walls': [
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod': 'InputUA',
          'direction': 'top', 'IsInContactWithOutsideAir': True, 'area': 67.8, 'space': 'main', 'UA': 0.24, 'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputAllDetails', 'direction': 'top', 
          'IsInContactWithOutsideAir': True, 'area': 67.8, 'space': 'main',
          'Parts': [{'AreaRatio': 0.8, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'AreaRatio': 0.2, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                  {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
          'IsSunshadeInput': False },
        { 'name': 'Floor', 'type': 'Floor', 'structure': 'wood', 'InputMethod' :'InputAllLayers', 'direction': 'bottom', 
          'IsInContactWithOutsideAir': True, 'area': 67.8, 'space': 'main',
          'FloorConstructionMethod' :'FrameBeamInsuljoist',
          'Parts': [{'TypeFloor': 'Insulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'TypeFloor': 'Heatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
           'IsSunshadeInput': False },
        { 'name': 'Wall', 'type': 'Wall', 'structure': 'wood', 'InputMethod' :'InputAllLayers', 'direction': 'N', 
          'IsInContactWithOutsideAir': True, 'area': 67.8, 'space': 'main', 
          'WallConstructionMethod': 'WallInsuladdBackvertical',
          'Parts': [{'TypeWall': 'WallInsulFilledandadd',        'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                                            {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'TypeWall': 'WallInsulfilledHeatbridgeadd', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                                            {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},  
                    {'TypeWall': 'WallInsuladdHeatbridgemember', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                                            {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},  
                    {'TypeWall': 'WallInsuladdHeatbridgelintel', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                                            {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},  
                    {'TypeWall': 'WallHeatbridgeMemberandadd'  , 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                                            {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},  
                    {'TypeWall': 'WallHeatbridgeLintelandadd'  , 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                                            {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],  
          'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputAllLayers', 'direction': 'top', 
          'IsInContactWithOutsideAir': True, 'area': 67.8, 'space': 'main', 
          'RoofConstructionMethod': 'Insulrafter', 
          'Parts': [{'TypeRoof': 'Insulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                          {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'TypeRoof': 'Heatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                          {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
          'IsSunshadeInput': False },
        { 'name': 'BoundaryCeiling', 'type': 'BoundaryCeiling', 'structure': 'wood', 'InputMethod' :'InputAllLayers', 
          'direction': 'top', 'IsInContactWithOutsideAir': False, 'area': 67.8, 'space': 'main', 'CeilingConstructionMethod': 'Insulbeam', 
          'Parts': [{'TypeCeiling': 'Insulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                             {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                    {'TypeCeiling': 'Heatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                             {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
          'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'wood', 'InputMethod' :'InputUR',
          'direction': 'top', 'IsInContactWithOutsideAir': False, 'area': 67.8, 'space': 'main', 'UR': 0.05,
          'Parts': [{'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
          'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'RC', 'InputMethod' :'InputUA', 
          'direction': 'top', 'IsInContactWithOutsideAir': True, 'area': 67.8, 'space': 'main', 'UA': 0.24, 'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'RC', 'InputMethod' :'InputLayers', 
          'direction': 'top', 'IsInContactWithOutsideAir': True, 'area': 67.8, 'space': 'main',
          'Parts': [{ 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                 {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],         
          'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'steel', 'InputMethod' :'InputUA', 
          'direction': 'top', 'IsInContactWithOutsideAir': True, 'area': 67.8, 'space': 'main', 'UA': 0.24, 'IsSunshadeInput': False },
        { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'steel', 'InputMethod' :'InputUR', 
          'direction': 'top', 'IsInContactWithOutsideAir': True, 'area': 67.8, 'space': 'main', 'Radding': 0.10, 
          'Parts': [{'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],         
          'IsSunshadeInput': False },
#         { 'name': 'Ceiling', 'type': 'Ceiling', 'structure': 'other', 'direction': 'top', 'area': 67.8, 'space': 'main', 
#           'IsSunshadeInput': False }
    ],
    'Windows': [
        { 'name': 'WindowSW', 'direction': 'SW', 'area': 30.25, 'space': 'main', 'UW': 6.51, 'TypeWindow': 'Single', 
          'IsEtaValueInput': False, 'TypeFrame': 'WoodOrResin', 'TypeGlass': '3WgG', 'TypeShade': 'Shoji',
          'IsSunshadeInput': True, 'Y1': 0.00, 'Y2': 1.00, 'Z': 0.60 },
        { 'name': 'WindowNW', 'direction': 'NW', 'area': 3.17, 'space': 'main', 'UW': 4.65, 'TypeWindow': 'Single', 
          'IsEtaValueInput': True, 'Eta': 0.738, 'IsSunshadeInput': False },
        { 'name': 'WindowSW', 'direction': 'SW', 'area': 30.25, 'space': 'main', 'UW': 6.51, 'TypeWindow': 'Double', 'IsEtaValueInput': False,
          'TypeFrameInside': 'WoodOrResin', 'TypeGlassInside': '3WgG', 'TypeShadeInside': 'Shoji',
          'TypeFrameOutside': 'Steel', 'TypeGlassOutside': '3WgG', 'TypeShadeOutside': 'ExtarnalBlind',
          'IsSunshadeInput': True, 'Y1': 0.00, 'Y2': 1.00, 'Z': 0.60 },
        { 'name': 'WindowNW', 'direction': 'NW', 'area': 3.17, 'space': 'main', 'UW': 4.65, 'TypeWindow': 'Double', 'IsEtaValueInput': True,
          'EtaInside': 0.738, 'TypeGlassInside': '3WgG', 'EtaOutside': 0.738, 'TypeGlassOutside': '3WgG',
          'IsSunshadeInput': True, 'Y1': 0.00, 'Y2': 1.00, 'Z': 0.60 }
    ],
    'Doors': [
        { 'name': 'DoorNW', 'direction': 'NW', 'area': 2.52, 'U': 6.51, 'space': 'main', 'IsSunshadeInput': False },
        { 'name': 'DoorNE', 'direction': 'NE', 'area': 2.16, 'U': 4.65, 'space': 'main', 
          'IsSunshadeInput': True, 'Y1': 0.00, 'Y2': 1.00, 'Z': 0.60 }
    ],
    'Heatbridges': [
        { 'name': 'NE', 'structure': 'RC', 'length': 1.00, 'psi': 1.8, 'direction1': 'N', 'direction2': 'E', 'space': 'main' },
        { 'name': 'NW', 'structure': 'Steel', 'length': 2.00, 'psi': 1.8, 'direction1': 'N', 'direction2': 'W', 'space': 'main' }
    ],
    'EarthfloorPerimeters': [
        { 'name': 'NW', 'direction': 'NW', 'length': 2.43, 'psi': 1.8, 'space': 'underfloor' },
        { 'name': 'NE', 'direction': 'NE', 'length': 2.43, 'psi': 1.8, 'space': 'underfloor' }
    ],
    'Earthfloors': [
        { 'name': 'other', 'area': 5.0, 'space': 'underfloor' },
        { 'name': 'entrance', 'area': 5.0, 'space': 'underfloor' }
    ],
    'InnerWalls': [
        {'area': 6.666666666666667,'direction': 'Horizontal', 'Layers': [{'cond': 0.16, 'name': 'PED', 'specH': 720.0, 'thick': 0.012}],
         'name': 'GroundFloor_other','spaces': ['other', 'underfloor'],'type': 'GroundFloor'},
        {'area': 53.333333333333336, 'direction': 'Horizontal', 'Layers': [{'cond': 0.16, 'name': 'PED', 'specH': 720.0, 'thick': 0.012}],
         'name': 'InnerFloor_other', 'nextspace': 'nonliving', 'space': 'other', 'type': 'InnerFloor'},
        {'area': 60.0, 'direction': 'Horizontal', 'Layers': [{'cond': 0.16, 'name': 'PED', 'specH': 720.0, 'thick': 0.012}],
         'name': 'InnerCeiling_other', 'nextspace': 'nonliving', 'space': 'other', 'type': 'InnerCeiling'},
        {'area': 121.45675773706458, 'direction': 'Vertical',
         'Layers': [{'cond': 0.22, 'name': 'GPB', 'specH': 830.0, 'thick': 0.0125},
                    {'cond': 11.11111111111111, 'name': 'GPB', 'specH': 0, 'thick': 1},
                    {'cond': 0.22, 'name': 'GPB', 'specH': 830.0, 'thick': 0.0125}],
         'name': 'Innerwall_other to nonliving', 'spaces': ['other', 'nonliving'], 'type': 'InnerWall'}
    ]
}

convert(d)

{'Common': {'IsSimplifiedInput': False,
  'MainOccupantRoomFloorArea': 30.0,
  'OtherOccupantRoomFloorArea': 60.0,
  'Region': 6,
  'TotalFloorArea': 120.0},
 'Doors': [{'IsFloor': False,
   'IsSunshadeInput': False,
   'U': 6.51,
   'area': 2.52,
   'direction': 'NW',
   'name': 'DoorNW',
   'space': 'main'},
  {'IsFloor': False,
   'IsSunshadeInput': True,
   'U': 4.65,
   'Y1': 0.0,
   'Y2': 1.0,
   'Z': 0.6,
   'area': 2.16,
   'direction': 'NE',
   'name': 'DoorNE',
   'space': 'main'}],
 'EarthfloorPerimeters': [{'direction': 'NW',
   'length': 2.43,
   'name': 'NW',
   'psi': 1.8,
   'space': 'underfloor'},
  {'direction': 'NE',
   'length': 2.43,
   'name': 'NE',
   'psi': 1.8,
   'space': 'underfloor'}],
 'Heatbridges': [{'direction1': 'N',
   'direction2': 'E',
   'length': 1.0,
   'name': 'NE',
   'psi': 1.8,
   'space': 'main',
   'structure': 'RC'},
  {'direction1': 'N',
   'direction2': 'W',
   'length': 2.0,
   'name': 'NW',
   'psi': 1.8,
   'space': 'main',
   'structu